In [ ]:
import os
import autogen
from dotenv import load_dotenv
from chromadb.utils import embedding_functions
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, inspect, MetaData
from langchain.text_splitter import RecursiveCharacterTextSplitter
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent, register_function

In [ ]:
load_dotenv("../.env")

LEAF_TABLE_NAME = "form_127"
LEAF_DB_NAME = "eform_data"
LEAF_DB_USER = "kamalleaf"
LEAF_DB_PASS = os.getenv("LEAF_DB_PASS")
LEAF_DB_HOST = "13.214.63.7"

llm_config = {
    "model": "gpt-4",
    "api_key": os.environ["OPENAI_API_KEY"],
    "temperature": 0,
}

Normal Chat - Getting Started

In [ ]:
# assistant = AssistantAgent(
#     name="assistant",
#     llm_config=llm_config,
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=5,
#     system_message="You are a smart chatbot, reply TERMINATE if you think you have answered user's question or if the user is sending empty message.",
#     is_termination_msg=lambda content: "TERMINATE" in content["content"]
#     or not content.get("content", ""),
# )
# user_proxy = UserProxyAgent(
#     name="user_proxy",
#     code_execution_config=False,
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=5,
#     system_message="You are going to ask question one time only, reply TERMINATE if you think your question has a reaction",
#     is_termination_msg=lambda content: "TERMINATE" in content["content"],
# )

# # Start the chat
# user_proxy.initiate_chat(
#     assistant,
#     message="My name is bob",
# )

For Chat history

In [ ]:
# for message in history:
#     user_proxy.send(message, assistant, request_reply=False)

In [ ]:
# class ResumableGroupChatManager(GroupChatManager):
#     groupchat: GroupChat

#     def __init__(self, groupchat, history, **kwargs):
#         self.groupchat = groupchat
#         if history:
#             self.groupchat.messages = history

#         super().__init__(groupchat, **kwargs)

#         if history:
#             self.restore_from_history(history)

#     def restore_from_history(self, history) -> None:
#         for message in history:
#             # broadcast the message to all agents except the speaker.  This idea is the same way GroupChat is implemented in AutoGen for new messages, this method simply allows us to replay old messages first.
#             for agent in self.groupchat.agents:
#                 if agent != self:
#                     self.send(message, agent, request_reply=False, silent=True)

For SQL query and reporting

In [ ]:
class LC_DB:
    def __init__(self) -> None:
        self.db = SQLDatabase.from_uri(
            f"mysql+mysqlconnector://{LEAF_DB_USER}:{LEAF_DB_PASS}@{LEAF_DB_HOST}/{LEAF_DB_NAME}",
            ignore_tables=set(
                ["eform_titles", "form_data_testing", "form_127", "form_170"]
            ),
        )

In [ ]:
lc_db = LC_DB()

In [ ]:
import json
import ast


def get_form_titles():
    query = lc_db.db.run("SELECT * FROM eform_titles")
    title_list = ast.literal_eval(query)
    return title_list

In [ ]:
from typing import Annotated, Literal

# Operator = Literal["form_131", "form_155", "form_156", "form_158", "form_171"]


def get_table_info(tablename: str) -> str:
    try:
        return lc_db.db.get_table_info([tablename])
    except Exception as e:
        return str(e)


def run_query(query: str) -> str:
    try:
        return lc_db.db.run(query)
    except Exception as e:
        return str(e)

In [ ]:
sql_writer = ConversableAgent(
    "query_writer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config=False,
    # code_execution_config={
    #     "executor": autogen.coding.LocalCommandLineCodeExecutor(work_dir="coding")
    # },
    system_message=f"""
    You are good at writing SQL queries.
    Below are the table names and their entities for you to lookup to.
    'form_131' - related to meeting minutes,
    'form_145' - related to Incident report,
    'form_155' - related to Water Supply Check List,
    'form_156' - related to Weekly Audio Visual Checklist - Ballroom SCCC1,
    'form_158' - related to Gym Room Checklist,
    'form_171' - related to Loan Form Setia
    'form_172' - related to Owner / Tenant Registration Form

    Write a SQL query based on the tables info returned by get_table_info()
    For each of these forms, take note of their correct references as below:
    1. 'form_131'
        - 'tf_MeetingMinuteSubject' identifies the type of meeting for this data entry. For example if user query for LEAF meeting, find a meeting where 'tf_MeetingMinuteSubject' == LEAF meeting
    Respond with TERMINATE after you have returned an aswer
    """,
    # is_termination_msg=check_termination,
    # is_termination_msg=lambda content: not content.get("content")
    # or "TERMINATE" in content.get("content"),
)

reporter = ConversableAgent(
    name="reporter",
    llm_config=llm_config,
    code_execution_config=False,
    system_message="""
    You have a data entry from database.
    Make sure to answer the user's question regarding the to the data.
    If the user enquiry summary, summarise them the content of the data
    return TERMINATE after you have all the relevant answer to return.
    """,
)

user_proxy = ConversableAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config=False,
    # code_execution_config={
    #     "executor": autogen.coding.LocalCommandLineCodeExecutor(work_dir="coding")
    # },
    is_termination_msg=lambda content: "TERMINATE" in content.get("content"),
)

register_function(
    get_table_info,
    caller=sql_writer,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="get_table_info",  # By default, the function name is used as the tool name.
    description="Use to describe a table info in a database",  # A description of the tool.
)

register_function(
    run_query,
    caller=sql_writer,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="run_sql_query",  # By default, the function name is used as the tool name.
    description="Use to run a sql command",  # A description of the tool.
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, sql_writer, reporter], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(
    manager, message="Summarise the 2nd LEAF meeting?", clear_history=True
)

For RAG agent

In [ ]:
# recur_spliter = RecursiveCharacterTextSplitter(separators=["\n", "\r", "\t"])

In [ ]:
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#     api_key=os.environ["OPENAI_API_KEY"],  # model_name="text-embedding-ada-002"
# )

In [ ]:
# assistant = RetrieveAssistantAgent(
#     name="assistant",
#     system_message="You are a helpful assistant.",
#     llm_config=llm_config,
# )

In [ ]:
# ragproxyagent = RetrieveUserProxyAgent(
#     name="ragproxyagent",
#     retrieve_config={
#         "task": "qa",
#         "docs_path": [
#             "../data/management/360/360_1715156298.pdf",
#             # "https://raw.githubusercontent.com/microsoft/autogen/main/README.md"
#         ],
#         "embedding_function": openai_ef,
#         "custom_text_split_function": recur_spliter.split_text,
#         "collection_name": "360_management",
#         "get_or_create": True,
#     },
#     code_execution_config=False,
#     human_input_mode="NEVER",
#     # code_execution_config={
#     #     "executor": autogen.coding.LocalCommandLineCodeExecutor(work_dir="coding")
#     # },
# )

In [ ]:
# ragproxyagent.initiate_chat(
#     assistant,
#     message=ragproxyagent.message_generator,
#     problem="How to set up alarm system?",
# )